In [3]:
from elasticsearch import Elasticsearch

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.indices.create("es0", body={
    "mappings": {
        "properties": {
            "name": {
                "type": "text",
                "analyzer": "ik_max_word"}}}})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'es0'}

In [4]:
es.index('es0', body={"name": "规范教材美术"}, id=0)
es.index('es0', body={"name": "规范教育化学"}, id=1)
es.index('es0', body={"name": "规范教育物理"}, id=2)
es.index('es0', body={"name": "规范教育数学"}, id=3)
es.index('es0', body={"name": "规范教授"}, id=4)
es.index('es0', body={"name": "规范教导学生"}, id=5)
es.index('es0', body={"name": "规范指导指引教育"}, id=6)
es.index('es0', body={"name": "规范指导学生"}, id=7)

{'_index': 'es0',
 '_type': '_doc',
 '_id': '7',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 7,
 '_primary_term': 1}

In [5]:
body = {
    "query": {
        "match_phrase_prefix": {
            "name": {
                # 查询内容"规划教"分词(ik_max_word)为: ['规范', '教.*'](即最后一个词的前缀为"教")
                # 文档索引条件:text类型字段分词结果包含且顺序为["规范", "教.*"]
                "query": "规范教",
                "boost": 1
            }
        }
    }
}
# Returns documents that contain the words of a provided text, in the same order as provided. The last term of the provided text is treated as a prefix, matching any words that begin with that term.
'''
GET test/_search
`body`
'''
es.search(index='es0', body=body)

{'took': 15,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 9.284731,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '4',
    '_score': 9.284731,
    '_source': {'name': '规范教授'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 8.049055,
    '_source': {'name': '规范教材美术'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 8.049055,
    '_source': {'name': '规范教育物理'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 8.049055,
    '_source': {'name': '规范教育数学'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '5',
    '_score': 8.049055,
    '_source': {'name': '规范教导学生'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 7.103652,
    '_source': {'name': '规范教育化学'}}]}}

In [6]:
'''
max_expansions
    (Optional, integer) Maximum number of terms to which the last provided term of the query value will expand. Defaults to 50.
'''
es.search(index='es0', body={"query":
                                 {"match_phrase_prefix":
                                      {"name":
                                           {"query": "规范教",
                                            # 匹配到的结果数(["规范", "教授"], ["规范", "教导"])为2
                                            "max_expansions": 2}}}})  # 前`max_expansions`个词可能匹配不到我们想要的结果(如"规范教育化学")

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 4.269451,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '4',
    '_score': 4.269451,
    '_source': {'name': '规范教授'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '5',
    '_score': 3.7012434,
    '_source': {'name': '规范教导学生'}}]}}

In [7]:
es.search(index='es0', body={"query":
                                 {"match_phrase_prefix":
                                      {"name":
                                           {"query": "规范教",
                                            # 匹配到的结果数(["规范", "教授"], ["规范", "教材"], ["规范", "教导"],  ["规范", "教育"])为4
                                            "max_expansions": 4}}}})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 5, 'relation': 'eq'},
  'max_score': 7.183521,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '4',
    '_score': 7.183521,
    '_source': {'name': '规范教授'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 6.2274885,
    '_source': {'name': '规范教材美术'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 6.2274885,
    '_source': {'name': '规范教育物理'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 6.2274885,
    '_source': {'name': '规范教育数学'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '5',
    '_score': 6.2274885,
    '_source': {'name': '规范教导学生'}}]}}

In [8]:
'''
分析:
"规范指导教育"的分词结果为:['规范', '指导', '指引', '教育']
["规范", "教.*"] no
["规范", "教.*"] "slop": 1 -> ["规范", ?, "教.*"] no
["规范", "教.*"] "slop": 2 -> ["规范", ?, ?, "教.*"] yes
'''
es.search(index='es0', body={"query":
                                 {"match_phrase_prefix":
                                      {"name":
                                           {"query": "规范教",
                                            "slop": 2  # 参数含义参考`match_phrase`
                                            }}}})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 7, 'relation': 'eq'},
  'max_score': 9.284731,
  'hits': [{'_index': 'es0',
    '_type': '_doc',
    '_id': '4',
    '_score': 9.284731,
    '_source': {'name': '规范教授'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '0',
    '_score': 8.049055,
    '_source': {'name': '规范教材美术'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '2',
    '_score': 8.049055,
    '_source': {'name': '规范教育物理'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '3',
    '_score': 8.049055,
    '_source': {'name': '规范教育数学'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '5',
    '_score': 8.049055,
    '_source': {'name': '规范教导学生'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '1',
    '_score': 7.103652,
    '_source': {'name': '规范教育化学'}},
   {'_index': 'es0',
    '_type': '_doc',
    '_id': '6',
    '_score': 3.2520819,
    '_source': {'name': 

In [9]:
es.indices.delete('es0')

{'acknowledged': True}